<a href="https://colab.research.google.com/github/guillaume117/ctf2023/blob/main/E1_basic_mat_mult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exercice 1 : MATRIX MULTIPLICATION WITH PARALLELIZED COMPUTATIONS IN A SINGLE BLOC

The goal of this first exercise is to get familiar with the basic concepts of GPU computing with CUDA. Note that the algorithms here won't be optimized nor computationally efficient... efficient strategies will start being studied in exercise 2.

-> Remark 1: Code adapted from: https://shephexd.github.io/development/2017/02/19/pycuda.html

-> Remark 2:  If using google colab:
* Click on Runtime (excecution) and select Change runtime type (modifier le type d'excecution).
* Then select GPU in Hardware Acceleration (accÃ©lÃ©rateur matÃ©riel)
* Start your session by installing pycuda with the command: \" !pip install pycuda \"


In [1]:
!pip install pycuda   #practical actually run on google colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.8 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2023.1-cp310-cp310-linux_x86_64.whl size=661263 sha256=c92345c528e7d3a090c5be2d0dd18016aa389e996e9e78db28566c36b6cba5d3
  Stored in directory: /root/.cache/pip/wheels/46/65/06/b997165edd2fd9690c3497ca54ea4485b571d7bd959c21c6c4
Successfully built pycuda


In [2]:
#+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + +
#1) INITIALISATION
#+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + +

import numpy as np
import pycuda
from pycuda import driver, compiler, gpuarray, tools
import time

# -- initialize the device
import pycuda.autoinit


#get device information
MyDevice=pycuda.driver.Device(0)
attributes=MyDevice.get_attributes()

for attribute in list(attributes.keys()):
  print(str(attribute)+": "+str(attributes[attribute]))


ASYNC_ENGINE_COUNT: 3
CAN_MAP_HOST_MEMORY: 1
CAN_USE_HOST_POINTER_FOR_REGISTERED_MEM: 1
CLOCK_RATE: 1590000
COMPUTE_CAPABILITY_MAJOR: 7
COMPUTE_CAPABILITY_MINOR: 5
COMPUTE_MODE: DEFAULT
COMPUTE_PREEMPTION_SUPPORTED: 1
CONCURRENT_KERNELS: 1
CONCURRENT_MANAGED_ACCESS: 1
DIRECT_MANAGED_MEM_ACCESS_FROM_HOST: 0
ECC_ENABLED: 1
GENERIC_COMPRESSION_SUPPORTED: 0
GLOBAL_L1_CACHE_SUPPORTED: 1
GLOBAL_MEMORY_BUS_WIDTH: 256
GPU_OVERLAP: 1
HANDLE_TYPE_POSIX_FILE_DESCRIPTOR_SUPPORTED: 1
HANDLE_TYPE_WIN32_HANDLE_SUPPORTED: 0
HANDLE_TYPE_WIN32_KMT_HANDLE_SUPPORTED: 0
HOST_NATIVE_ATOMIC_SUPPORTED: 0
INTEGRATED: 0
KERNEL_EXEC_TIMEOUT: 0
L2_CACHE_SIZE: 4194304
LOCAL_L1_CACHE_SUPPORTED: 1
MANAGED_MEMORY: 1
MAXIMUM_SURFACE1D_LAYERED_LAYERS: 2048
MAXIMUM_SURFACE1D_LAYERED_WIDTH: 32768
MAXIMUM_SURFACE1D_WIDTH: 32768
MAXIMUM_SURFACE2D_HEIGHT: 65536
MAXIMUM_SURFACE2D_LAYERED_HEIGHT: 32768
MAXIMUM_SURFACE2D_LAYERED_LAYERS: 2048
MAXIMUM_SURFACE2D_LAYERED_WIDTH: 32768
MAXIMUM_SURFACE2D_WIDTH: 131072
MAXIMUM_SURFACE

In [15]:

#+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + +
#2) MATRIX MULTIPLICATION ON THE CPU
#+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + +

# define the (square) matrix size
#  note that we'll only use *one* block of threads here
#  as a consequence this number (squared) can't exceed max_threads
# -> use MyDevice.get_attributes() to get this information
MATRIX_SIZE = 128

# create two random square matrices
a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)

# compute reference on the CPU to verify GPU computation
time_start=time.time()
c_cpu = np.dot(a_cpu, b_cpu)
time_end=time.time()
print('enlapsed time (CPU):',time_end-time_start,' seconds')



enlapsed time (CPU): 0.004463911056518555  seconds


In [16]:

#+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + +
#3) MAKE AND COMPILE THE KERNEL FOR MATRIX MULTIPLICATION ON THE GPU
#+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + +

#define the kernel (for the meaning of %()s, see: https://stackoverflow.com/questions/63862118/what-is-the-meaning-of-s-in-python)
kernel_code_templates = """
__global__ void MatrixMulKernel_sequential(float *a, float *b, float *c)
{
    for (int i = 0; i < %(MATRIX_SIZE)s; ++i) {
      for (int j = 0; j < %(MATRIX_SIZE)s; ++j) {
        c[i * %(MATRIX_SIZE)s + j] = 0.;
        for (int k = 0; k < %(MATRIX_SIZE)s; ++k) {
          c[i * %(MATRIX_SIZE)s + j] += a[i * %(MATRIX_SIZE)s + k] * b[k * %(MATRIX_SIZE)s + j];
        }
      }
    }
}
__global__ void MatrixMulKernel_sequential2(float *a, float *b, float *c)
{
    float c_value;

    for (int i = 0; i < %(MATRIX_SIZE)s; ++i) {
      for (int j = 0; j < %(MATRIX_SIZE)s; ++j) {
        c_value = 0.;
        for (int k = 0; k < %(MATRIX_SIZE)s; ++k) {
          c_value += a[i * %(MATRIX_SIZE)s + k] * b[k * %(MATRIX_SIZE)s + j];
        }
        c[i * %(MATRIX_SIZE)s + j]=c_value;
      }
    }
}
__global__ void MatrixMulKernel_parallel(float *a, float *b, float *c)
{
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // Pvalue is used to store the element of the matrix
    // that is computed by the thread
    float Pvalue = 0;

    // Each thread loads one row of M and one column of N,
    //   to produce one element of P.
    for (int k = 0; k < %(MATRIX_SIZE)s; ++k) {
        Pvalue += a[ty * %(MATRIX_SIZE)s + k] * b[k * %(MATRIX_SIZE)s + tx];
    }

    // Write the matrix to device memory;
    // each thread writes one element
    c[ty * %(MATRIX_SIZE)s + tx] = Pvalue;
}
"""

# get the kernel code from the template
# by specifying the constant MATRIX_SIZE
kernel_codes = kernel_code_templates % {
    'MATRIX_SIZE': MATRIX_SIZE
    }

# compile the kernel code
mod = compiler.SourceModule(kernel_codes)

# get the kernel function from the compiled module
matrixmul_sequential = mod.get_function("MatrixMulKernel_sequential")
matrixmul_sequential2 = mod.get_function("MatrixMulKernel_sequential2")
matrixmul_parallel = mod.get_function("MatrixMulKernel_parallel")


In [17]:

#+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + +
#4) MATRIX MULTIPLICATION ON THE GPU
#+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + +


# transfer host (CPU) memory to device (GPU) memory
a_gpu = gpuarray.to_gpu(a_cpu)
b_gpu = gpuarray.to_gpu(b_cpu)

# create empty gpu array for the result (C = A * B)
c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)



# call the kernel on the card -- sequential matrix multiplication
time_start=time.time()
matrixmul_sequential(a_gpu , b_gpu , c_gpu , block = (1, 1, 1))
time_end=time.time()

print('enlapsed time (GPU - sequential):',time_end-time_start,' seconds')
print('norm(c_cpu-c_gpu)='+str(np.linalg.norm(c_cpu - c_gpu.get())))

# call the kernel on the card -- sequential (slighly improved) matrix multiplication
time_start=time.time()
matrixmul_sequential2(a_gpu , b_gpu , c_gpu , block = (1, 1, 1))
time_end=time.time()

print('enlapsed time (GPU - sequential 2):',time_end-time_start,' seconds')
print('norm(c_cpu-c_gpu)='+str(np.linalg.norm(c_cpu - c_gpu.get())))


# call the kernel on the card
time_start=time.time()
matrixmul_parallel(a_gpu, b_gpu, c_gpu, block = (MATRIX_SIZE, MATRIX_SIZE, 1))
time_end=time.time()
print('enlapsed time (GPU - parallel):',time_end-time_start,' seconds')
print('norm(c_cpu-c_gpu)='+str(np.linalg.norm(c_cpu - c_gpu.get())))




enlapsed time (GPU - sequential): 0.00022912025451660156  seconds
norm(c_cpu-c_gpu)=0.0
enlapsed time (GPU - sequential 2): 0.0001773834228515625  seconds
norm(c_cpu-c_gpu)=0.0


/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()


LogicError: ignored

##QUESTION 1
Comprenez bien chaque partie du code en vous attardant sur :
- 1.1 :les commandes utilisees pour ecrire, compiler et executer les differents noyaux (kernels)
- 1.2 : le lien entre les algorithmes implÃ©mentÃ©s dans les differents noyaux utilises et la taille des blocs lors de leur appel.

##QUESTION 2
Comparez les temps de calucul entre la multiplication CPU et celles GPU (relancez plusieurs fois les calculs car les temps d'execution peuvent etres trÃ¨s variables d'une fois sur l'autre en fonction de l'utilisation du PC/serveur. Gardez alors les temps les plus courts). Il est tout a fait possible que les gains ne soient pas bons (ce sera moins le cas dans les exercices 2 et 3 avec des algorithmes plus adaptes au GPU). Pourquoi ?
             
##QUESTION 3
Comparez les temps de calculs entre la methode 'sequential' et celle 'sequential 2'. Pourquoi la seconde est un peu plus rapide ?
             
##QUESTION 4
Comparez les temps de calculs entre la methode 'sequential' et celle 'parallel'. Pourquoi les gains sont si faibles (voir negatifs) alors que les calculs on ete parallelises sur 1024 noeuds ?
             
##QUESTION 5
Il est trÃ¨s clair que les calculs parallÃ©lisÃ©s sur chaque noeud sont trop court avec une matrice de taille 32x32 par rapport au temps perdu a deplacer l'information et a ordonancer la parallelisation. Relancez alors les calculs avec une taille de matrice plus grande (disons 128x128). Pourquoi cela ne marche pas ?

La rÃ©ponse va en tous cas nous motiver a utiliser dans l'exercice 2 une grille de blocs et non un seul bloc... ce qui va rendre les gains de temps bien plus interessants !


Q1 - OK
Q2 - Le gain est limité par la dimension de la matrice et le besoin de faire un appel mémoire sur le GPU. Le CPU est donc plus performant pour un calcul simple
Q3

Q3 - Sequential 2 fait moins d'appel mémoire

Q4 - Les gains sont faible car temps accès mémoire limitant
Q5 - On dépasse la taille maximale de thread pour paralléliser le calcul